In [ ]:
!guardrails hub install hub://guardrails/two_words

## Input Validation

Guardrails supports validating inputs (prompts, instructions, msg_history) with string validators.

In XML, specify the validators on the `prompt` or `instructions` tag, as such:

In [4]:
rail_spec = """
<rail version="0.1">
<prompt
    validators="two-words"
    on-fail-two-words="exception"
>
This is not two words
</prompt>
<output type="string">
</output>
</rail>
"""

from guardrails import Guard
guard = Guard.from_rail_string(rail_spec)

When `fix` is specified as the on-fail handler, the prompt will automatically be amended before calling the LLM.

In any other case (for example, `exception`), a `ValidationError` will be returned in the outcome.

In [5]:
import openai
from guardrails.errors import ValidationError

try:
    guard(
        openai.chat.completions.create,
    )
except ValidationError as e:
    print(e)

Validation failed for field with errors: must be exactly two words


When using pydantic to initialize a `Guard`, input validators can be specified by composition, as such:

In [6]:
from guardrails.hub import TwoWords
from pydantic import BaseModel


class Pet(BaseModel):
    name: str
    age: int


guard = Guard.from_pydantic(Pet)
guard.with_prompt_validation([TwoWords(on_fail="exception")])

try:
    guard(
        openai.chat.completions.create,
        prompt="This is not two words",
    )
except ValidationError as e:
    print(e)

Validation failed for field with errors: must be exactly two words
